## Download the data

In [1]:
import os.path
if not os.path.isfile('data/vmware_ir_content.csv'):
    !pip install kaggle
    !kaggle competitions download -c vmware-zero-shot-information-retrieval
    !mkdir -p data/
    !unzip -o vmware-zero-shot-information-retrieval.zip
    !mv *.csv data/

## Parse the vmware corpus

- Load vmware text
- Read metadata into fields

In [40]:
import pandas as pd
corpus = pd.read_csv('data/vmware_ir_content.csv')
queries = pd.read_csv('data/test.csv')
corpus = corpus.fillna('')

import json
import ast

parsed_rows = []

for row in corpus.to_dict(orient='records'):
    row_dict = ast.literal_eval(row['raw_meta'])
    row_dict['id'] = row['f_name']
    row_dict['f_name'] = row['f_name']
    row_dict['raw_text'] = row['raw_text']
    row_dict['document_group'] = row['document_group']

    # fields that dont index for some reason
    bad_fields = ["DC.Date", "DC.Publisher"]
    for bad_field in bad_fields:
        if bad_field in row_dict:
            del row_dict[bad_field]
        
    parsed_rows.append(row_dict)
    
corpus = pd.DataFrame(parsed_rows)
corpus

,onetrust-data-domain,microsites-utag,titleTag,csrf-param,csrf-token,DC.Title,DC.Subject,DC.Language,google-site-verification,facebook-domain-verification,...,machine-translated,name,custom_label,link_url,children,url,lastModified,md5,publication-author-name,output-file-name
0,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Black Duck for VMware Tanzu | Solutions Hub | ...,authenticity_token,NnLorKoXWVAXtOGcZgn6cD0qO7KbHyuGc6QwI0FaGybXtO...,Black Duck for VMware Tanzu | Solutions Hub,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Percona | Solutions Hub | VMware Tanzu,authenticity_token,2N9nj_cHWOjsFUJe2rbR8Rlllj9DfNaD9m79W4GYB9IlnP...,Percona | Solutions Hub,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,United States Air Force | Customers | VMware ...,authenticity_token,CCmw9wy9YaWEur35yHQYIvF6XbSJ23oUmZsF2qf5Qmyo83...,United States Air Force | Customers,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Dynatrace for VMware Tanzu | Solutions Hub | V...,authenticity_token,VqAn3GNy9pSHWoDrlxi_mxbAM6XrBFTcIDQeYUxvWCehtX...,Dynatrace for VMware Tanzu | Solutions Hub,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,New Relic for VMware Tanzu | Solutions Hub | V...,authenticity_token,VM0vueZ5tE_2BnTwD-f09O5A3bKGe-eCZO2ixNeHAzd-hW...,New Relic for VMware Tanzu | Solutions Hub,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 

In [63]:
#for row in corpus[corpus['id'].str.contains('GUID')]['id']:
#    print(row)

corpus

,onetrust-data-domain,microsites-utag,titleTag,csrf-param,csrf-token,DC.Title,DC.Subject,DC.Language,google-site-verification,facebook-domain-verification,...,machine-translated,name,custom_label,link_url,children,url,lastModified,md5,publication-author-name,output-file-name
0,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Black Duck for VMware Tanzu | Solutions Hub | ...,authenticity_token,NnLorKoXWVAXtOGcZgn6cD0qO7KbHyuGc6QwI0FaGybXtO...,Black Duck for VMware Tanzu | Solutions Hub,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Percona | Solutions Hub | VMware Tanzu,authenticity_token,2N9nj_cHWOjsFUJe2rbR8Rlllj9DfNaD9m79W4GYB9IlnP...,Percona | Solutions Hub,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,United States Air Force | Customers | VMware ...,authenticity_token,CCmw9wy9YaWEur35yHQYIvF6XbSJ23oUmZsF2qf5Qmyo83...,United States Air Force | Customers,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Dynatrace for VMware Tanzu | Solutions Hub | V...,authenticity_token,VqAn3GNy9pSHWoDrlxi_mxbAM6XrBFTcIDQeYUxvWCehtX...,Dynatrace for VMware Tanzu | Solutions Hub,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,New Relic for VMware Tanzu | Solutions Hub | V...,authenticity_token,VM0vueZ5tE_2BnTwD-f09O5A3bKGe-eCZO2ixNeHAzd-hW...,New Relic for VMware Tanzu | Solutions Hub,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
from ltr.client import ElasticClient
client=ElasticClient()

from ltr.index import rebuild
rebuild(client, index='vmware', doc_src=parsed_rows, force=True)

Deleted index vmware [Status: 200]
Created index vmware [Status: 200]
Streaming Bulk index DONE vmware [Status: 201]


In [86]:
def exact_phrase_title(client, query):
    es = client.es
    body = {
        'size': 5,
        'query': {
            'multi_match': {
                'fields': ['title'],
                'type': 'phrase',
                'query': query
            }
        }
    }
    return es.search(index='vmware', body=body)['hits']['hits']


def exact_phrase(client, query):
    es = client.es
    body = {
        'size': 5,
        'query': {
            'multi_match': {
                'fields': ['title^100', 'description', 'raw_text'],
                'type': 'phrase',
                'query': query
            }
        }
    }
    return es.search(index='vmware', body=body)['hits']['hits']

In [91]:
def search(client, query):
    es = client.es
    body = {
        'size': 5,
        'query': {
            'multi_match': {
                'fields': ['title', 'description', 'raw_text'],
                'type': 'cross_fields',
                'query': query
            }
        }
    }
    return es.search(index='vmware', body=body)['hits']['hits']


def search_all(client, queries, how=search):
    all_results = []
    for query in queries.to_dict(orient='records'):
        print(f"Searching for {query['QueryId']},{query['Query']}")
        results = how(client, query['Query'])
        for rank, result in enumerate(results):
            source = result['_source']
            if rank == 0 and 'titleTag' in source:
                print(source['titleTag'])
            source['rank'] = rank
            source['score'] = result['_score']
            source['DocumentId'] = source['id']
            source['QueryId'] = query['QueryId']
            all_results.append(source)
        print()
        print()
    all_results = pd.DataFrame(all_results)
    return queries.merge(all_results, how='left', on='QueryId').sort_values(['QueryId', 'rank'])
    
#submission = search_all(client, queries)

In [92]:
exact_phrase_matches = search_all(client, queries, how=exact_phrase)

Searching for 0,what is hypervisor


Searching for 1,what is a hypervisor
What is a Hypervisor? | VMware Glossary


Searching for 2,what is vmware


Searching for 3,unable to login because you do not have permission on any vcenter server systems connected to this client.
Troubleshooting permissions errors when connecting to vCenter Server with the vSphere Client (1003872) | VMware KB


Searching for 4,what is virtualization
What is virtualization technology& virtual machine? | VMware


Searching for 5,what is a virtual machine


Searching for 6,how to install vmware tools ubuntu


Searching for 7,how to use vmware
Become a Threat Hunter | VMware Carbon Black


Searching for 8,how to install vmware tools
How to install VMware tools on FreeBSD server - VMware Customer Experience and Success


Searching for 9,how to install vmware tools linux


Searching for 10,unable to login because you do not have permission on any vcenter server systems connected to this client
Troubleshooting permiss



Searching for 106,what is vmware esxi


Searching for 107,what is vmotion in vmware


Searching for 108,what is cyber espionage
What is Cyber Espionage | VMware Glossary


Searching for 109,how to check if elasticsearch is running


Searching for 110,how to change esxi hostname


Searching for 111,what are hypervisors


Searching for 112,what is a namespace in kubernetes


Searching for 113,how to access esxi shell


Searching for 114,how to check vmware license expiry date


Searching for 115,an error occurred while sending an authentication request to the vcenter single sign-on server - an error occurred when processing metadata during vcenter single sign-on setup: the service provider validation failed. verify that the server url is correct and is in fqdn format, or that the hostname is a trusted service provider alias.


Searching for 116,how to upgrade vmware tools


Searching for 117,what is cloud native
Cloud Native Applications: State of the Union - Cloud Native Apps Blog


S

Linux virtual machine fails to respond after a mounted CDROM is disconnected (2144053) | VMware KB


Searching for 208,what is vmware player
VMware Player Plus FAQs: Create and run virtual machines on a Windows or Linux PC | United States


Searching for 209,how to create a windows 10 vm in vmware


Searching for 210,how to disable relative mouse in vmware


Searching for 211,how to upgrade vcenter 6.5 to 7.0


Searching for 212,what is vrealize suite
What is vRealize Suite& vCloud Suite? | VMware


Searching for 213,how to check vmware version


Searching for 214,unable to login because you do not have permission on any vcenter
Troubleshooting permissions errors when connecting to vCenter Server with the vSphere Client (1003872) | VMware KB


Searching for 215,how to unmount datastore from esxi host


Searching for 216,what is vcenter server in vmware


Searching for 217,an error occurred when processing the metadata during vcenter single sign-on setup
"An error occurred while sending



Searching for 318,what does disaster recovery meaning


Searching for 319,how to open a vmdk file in vmware


Searching for 320,how to install vmtools on linux


Searching for 321,how to increase vm disk size


Searching for 322,you do not have permission to enter a license key vmware


Searching for 323,how to restart hostd service in esxi


Searching for 324,an error occurred while sending an authentication request to the vcenter single sign-on server - an error occurred when processing the metadata during vcenter single sign-on setup - cannot connect to the vmware component manager


Searching for 325,how to install windows 10 on vmware fusion


Searching for 326,how to uninstall vmware on mac


Searching for 327,how to disable time sync between host and guest vmware


Searching for 328,how to configure vsan
How to configure vSAN VMkernel networking (2058368) | VMware KB


Searching for 329,how to reinstall vmware tools


Searching for 330,how to block bluetooth device


Searching



Searching for 420,how does a hypervisor work
What is a Hypervisor? | VMware Glossary


Searching for 421,how to install windows 10 in vmware workstation 15


Searching for 422,how to upgrade esxi host
How to upgrade ESXi hosts in VMware Cloud Foundation 3.5 with a vendor-specific ISO image (65047) | VMware KB


Searching for 423,how to restart vm


Searching for 424,what is virtualization in networking


Searching for 425,what is mobile security


Searching for 426,what does intelligent hub track


Searching for 427,what is desktop as a service
What is Desktop as a Service (DaaS)? | VMware Glossary


Searching for 428,how to install vmware in linux


Searching for 429,what is vmware workspace one


Searching for 430,how to create vm from vmdk file


Searching for 431,what are cloud native applications
What are Cloud Native Applications? | VMware Tanzu


Searching for 432,what is fault tolerance in vmware


Searching for 433,where are vmware files stored windows 10


Searching for 434

...



Searching for 1792,how to take screenshot in vmware horizon client


Searching for 1793,"unable to login because you do not have permission on any vcenter server systems connected to this client"
Troubleshooting permissions errors when connecting to vCenter Server with the vSphere Client (1003872) | VMware KB


Searching for 1794,how to upgrade vmware horizon


Searching for 1795,how to clone virtual machine


Searching for 1796,how to delete vm from vmware


Searching for 1797,how to mount vmware tools in linux


Searching for 1798,how to check vmkernel logs esxi 6.7


Searching for 1799,how to copy file from vmware to local machine


Searching for 1800,what is container deployment


Searching for 1801,how to copy files from vmware to local machine


Searching for 1802,how to use vmware workstation 15
VMware Workstation 15.x Pro sales, licensing, and compatibility FAQs (56987) | VMware KB


Searching for 1803,what is the purpose of a hypervisor server


Searching for 1804,how to res



Searching for 1919,what is the difference between thin and thick provisioning vmware


Searching for 1920,how to change keyboard language in vmware console


Searching for 1921,how much is vmware


Searching for 1922,what is ui mode


Searching for 1923,if the host is part of a vsphere ha cluster, the automatic startup and shutdown of virtual machines is disabled.


Searching for 1924,what vmware
VMworld 2020: What’s New with Workspace ONE Unified Endpoint Management


Searching for 1925,how to enable shortcuts on mac


Searching for 1926,how to enable copy paste in vmware


Searching for 1927,how is nsx licensed


Searching for 1928,how much ram to allocate to virtual machine


Searching for 1929,how to connect to esxi


Searching for 1930,how to disable virtualization based security windows 10


Searching for 1931,how to secure vmware environment


Searching for 1932,what does virtualisation do


Searching for 1933,how to change password in vm


Searching for 1934,how to connect to



Searching for 2042,how to check elasticsearch is running or not in linux


Searching for 2043,how to close vmware


Searching for 2044,what is virtual machine image


Searching for 2045,what is the core component of intent-based networking


Searching for 2046,what is fault symptoms


Searching for 2047,what is multipathing in vmware


Searching for 2048,what is the difference between vmware fusion and fusion pro?


Searching for 2049,how to migrate a vm from one datacenter to another


Searching for 2050,what is san in computer


Searching for 2051,how to install vmware in ubuntu using terminal


Searching for 2052,why virtual machine is slow


Searching for 2053,how to exit vmware full screen


Searching for 2054,how to view vmware logs


Searching for 2055,check if ssh daemon is running


Searching for 2056,how to configure iscsi in vmware esxi 6.7


Searching for 2057,what is vmware cloud


Searching for 2058,what vmware do i need


Searching for 2059,how to connect to vm


Searc



Searching for 2178,how to move vm from one vcenter to another


Searching for 2179,how to expand vmware disk size


Searching for 2180,how to use vsphere
Unable to upload custom patches to the patch repository of the VUM with error "Failed to upload file" (65043) | VMware KB


Searching for 2181,how to know vmware version


Searching for 2182,what is hostd in vmware


Searching for 2183,how to turn off virtual machine


Searching for 2184,how to remove hub app


Searching for 2185,how to install vmware tools in debian


Searching for 2186,what does it mean to be cloud native


Searching for 2187,what is deployment kubernetes


Searching for 2188,vsan health alarm 'vsphere cluster members do not match vsan cluster members'


Searching for 2189,how to upgrade vsan


Searching for 2190,how vm works


Searching for 2191,how to turn off relative mouse


Searching for 2192,what is evc mode in vmware


Searching for 2193,how to setup vsphere
VMware Communities Roundtable Podcast 238 - IT Zo



Searching for 2333,how to check vmtools version in windows


Searching for 2334,how to set the execution policy in powershell


Searching for 2335,what is sts certificate


Searching for 2336,what tasks do hypervisors accomplish in cloud computing


Searching for 2337,what is a minimum disk controller requirement for vmware vsan?


Searching for 2338,how to watch youtube fullscreen dual monitor


Searching for 2339,what does remote first mean


Searching for 2340,how to login to vmware host


Searching for 2341,how to upgrade vmware workstation


Searching for 2342,how to install powercli offline


Searching for 2343,what is dhcp binding


Searching for 2344,how to mount shared folder in linux


Searching for 2345,what is vmware fusion pro


Searching for 2346,vsphere cluster members do not match vsan cluster members


Searching for 2347,how to create a content library
Managing Horizon Gold Images Across Multi-Site Deployments - VMware Blogs


Searching for 2348,how to restore vmdk f

In [94]:
exact_phrase_matches[(exact_phrase_matches['score'] > 20) & 
                     (~exact_phrase_matches['titleTag'].isna())][['Query', 'titleTag', 'score']]

,Query,titleTag,score
1,what is a hypervisor,What is a Hypervisor? | VMware Glossary,20.013910
7,unable to login because you do not have permis...,Troubleshooting permissions errors when connec...,27.604921
30,unable to login because you do not have permis...,Troubleshooting permissions errors when connec...,27.604921
46,what is kubernetes cluster,What is Kubernetes Cluster? | VMware Glossary,20.657581
47,what is hyperconverged infrastructure,What is Hyperconverged Infrastructure? | 3 Ben...,20.200089
...,...,...,...
2664,what is hybrid cloud,What is Hybrid Cloud Management? | VMware Glos...,20.548573
2665,what is hybrid cloud,What is Hybrid Cloud Computing? | VMware Glossary,20.548573
2679,what is disaster recovery as a service?,What is Disaster Recovery as a Service (DRaaS)...,25.255249
2728,what is application modernization,What is Application Modernization? | VMware Gl...,25.576553


In [70]:
def write_submission():
    from time import time
    timestamp = str(time()).replace('.', '')
    fname = f'data/turnbull_submission_{timestamp}.csv'
    print("Writing To: ", fname)
    submission[['QueryId', 'DocumentId']].to_csv(fname, index=False)

In [71]:
submission[['QueryId', 'DocumentId']]

,QueryId,DocumentId
0,0,https---www.vmware.com-topics-glossary-content...
1,0,https---www.vmware.com-products-vsphere-hyperv...
2,0,https---www.vmware.com-topics-glossary-content...
3,0,https---www.vmware.com-topics-glossary-content...
4,0,https---www.vmware.com-topics-glossary-content...
...,...,...
11750,2350,https---www.vmware.com-topics-glossary-content...
11751,2350,en-2021-vRealize-Automation-8-3-Using-and-Mana...
11752,2350,en-2021-vRealize-Automation-8-4-Using-and-Mana...
11753,2350,en-2021-VMware-Cloud-Assembly-services-Using-a...


In [72]:
write_submission()

Writing To:  data/turnbull_submission_1652544680901428.csv


In [57]:
corpus[corpus['id'].str.contains('https---blogs.vmware.com-cloudprovider-2015-11-simplifying-cloud-spending-with-vmware-subscription-purchase-program.txt')]

,onetrust-data-domain,microsites-utag,titleTag,csrf-param,csrf-token,DC.Title,DC.Subject,DC.Language,google-site-verification,facebook-domain-verification,...,machine-translated,name,custom_label,link_url,children,url,lastModified,md5,publication-author-name,output-file-name
11041,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Simplifying Cloud Spending with VMware Subscri...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
